<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings("ignore")

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [34]:
# текст запроса
query_3_1 = f'''SELECT COUNT(v.id) 
FROM public.vacancies v'''


In [35]:
# результат запроса

result = pd.read_sql_query(query_3_1, connection)
print('Общее число вакансий в базе данных составляет: ', result.iloc[0, 0])


Общее число вакансий в базе данных составляет:  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [36]:
# текст запроса

query_3_2 = f'''SELECT COUNT(e.id) 
FROM public.employers e'''

In [37]:
# результат запроса

result = pd.read_sql_query(query_3_2, connection)
print('Количество работадателей в базе данных составляет: ', result.iloc[0, 0])

Количество работадателей в базе данных составляет:  23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [38]:
# текст запроса

query_3_3 = f'''SELECT COUNT(a.id) 
FROM public.areas a'''

In [39]:
# результат запроса

result = pd.read_sql_query(query_3_3, connection)
print(f"В базе данных встречаюся {result.iloc[0, 0]} региона.")

В базе данных встречаюся 1362 региона.


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [40]:
# текст запроса

query_3_4 = f'''SELECT COUNT(i.id) 
FROM public.industries i'''

In [41]:
# результат запроса

result = pd.read_sql_query(query_3_4, connection)
print(f"В базе данных представлены {result.iloc[0, 0]} сферы деятельности")

В базе данных представлены 294 сферы деятельности


***

***Комментарии и выводы:***

- *На этапе предварительного анализа можно заметить довольно значительное количество работадателей и всего в 2 раза больше число вакансий, но также стоит отметить, что не все работадатели, сферы деятельности и территориальные зоны представлены в нашем конкретном перечне вакансий (в таблице vacancies)*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [42]:
# текст запроса

query_4_1 = f'''SELECT a.name,
COUNT(v.id) -- считаем количество вакансий
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id --соединяем таблицы вакансий и регионов по идентификаторам
GROUP BY a.name -- группируем по названием региона
ORDER BY COUNT(v.id) DESC -- сортируем по количеству вакансий в порядке убывания
LIMIT 5 -- ограничиваем вывод 5-ю первыми значениями'''

In [43]:
# результат запроса

result = pd.read_sql_query(query_4_1, connection)
result_list = list(result.loc[:, 'name'])
print(f"Пятёрку лидеров по количеству вакансий возглавляют: {result_list}")

Пятёрку лидеров по количеству вакансий возглавляют: ['Москва', 'Санкт-Петербург', 'Минск', 'Новосибирск', 'Алматы']


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [44]:
# текст запроса

query_4_2 = f''' SELECT COUNT(v.id)
FROM public.vacancies v
WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL -- условия не равенства пустоте хотя бы у одного поля'''

In [45]:
# результат запроса

result = pd.read_sql_query(query_4_2, connection)
print ('Количества вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой, составляют: ', result.iloc[0, 0])

Количества вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой, составляют:  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [46]:
# текст запроса

query_4_3 = f''' SELECT ROUND(AVG (v.salary_from), 0) AVG_for_the_lower_limit, --считаем среднее значение для нижней границы зарплатной вилки
ROUND(AVG(v.salary_to), 0) AVG_for_the_upper_limit--считаем среднее значение для верхней границы зарплатной вилки
FROM public.vacancies v'''

In [47]:
# результат запроса

result = pd.read_sql_query(query_4_3, connection)
print('Средние значения для нижней границы зарплатной вилки составляет', result.loc[0,'avg_for_the_lower_limit'])
print('Средние значения для верхней границы зарплатной вилки составляет', result.loc[0,'avg_for_the_upper_limit'])

Средние значения для нижней границы зарплатной вилки составляет 71065.0
Средние значения для верхней границы зарплатной вилки составляет 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [48]:
# текст запроса

query_4_4 = f''' SELECT v.schedule ||' - '|| v.employment "schedule - employment", -- находим сочетиния типа рабочего графика и типа трудоустройства
COUNT(v.schedule ||' - '|| v.employment) -- считаем количество вакансий из полученных сочетаний
FROM public.vacancies v
GROUP BY v.schedule ||' - '|| v.employment -- группируем по полученным сочетаниям
ORDER BY COUNT(v.schedule ||' - '|| v.employment) DESC
OFFSET 1 LIMIT 1'''

In [49]:
# результат запроса

result = pd.read_sql_query(query_4_4, connection)
print('На втором месте по популярности находится сочетание типа рабочего графика и типа трудоустройства: ', result.iloc[0, 0])


На втором месте по популярности находится сочетание типа рабочего графика и типа трудоустройства:  Удаленная работа - Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [50]:
# текст запроса

query_4_5 = f''' SELECT v.experience --, 
--COUNT(v.id) -- считаем количество вакансий по требуемому опыту работы
FROM public.vacancies v
GROUP BY v.experience -- группируем по требуемому опыту работы
ORDER BY COUNT(v.id) ASC'''

In [51]:
# результат запроса

result = pd.read_sql_query(query_4_5, connection)
result_list = list(result['experience'])
print('Взаимосвязь требуемого опыта работы от количества вакансий приведена в следующем возрастающем порядке: ', result_list)

Взаимосвязь требуемого опыта работы от количества вакансий приведена в следующем возрастающем порядке:  ['Более 6 лет', 'Нет опыта', 'От 3 до 6 лет', 'От 1 года до 3 лет']


***

***Комментарии и выводы:***

*В детальном анализе вакансий заметны следующие тенденции:*

- *Пятёрка лидеров по количеству вакансий подтверждает неоспаримую особенность рынка труда в больших и огромных городах (мегаполисах). Cуществует довольно большое количество предложений по трудоустройству (большое количество разных предприятий, разных представительств крупных фирм, торговых площадок, тц, большое количество людей в сфере обслуживания и т.д.), существует конкуренция между работодателями, а у работника есть выбор кому продать свои знания и умения подороже.*

- *Чуть больше половины вакансий имеют заполненные значения хотя бы в одном из двух полей с возможной наименьшей и наибольшей зарплатой по данной вакансии. Это может означать не совсем открытую игру работодателя перед соискателем (предложением "серых зарплат"), но возможно и желанием скрыть данную информацию от различных структур и конкурентов.*

- *Среднее значение для нижней границы зарплат и для  верхней границы зарплат по вакансиям, указывает нам о достойных заработных платах, причем верхний предел зарплат в 1,5 раза больше нижнего.*

- *На втором месте по популярности находится сочетание типа рабочего графика и типа трудоустройства:  Удаленная работа - Полная занятость. А самый популярный вариант сочетания: Полный день - Полная занятость. Это преобладающая тендеция работодателей, что работники должны быть в офисе. Но данная тенденция возможно будет изменяться (количество вакансий в 4,5 раза больше на первом месте).*

- *Наибольшее количество вакансий представлена с опытом работы от 1 года до 3 лет. Видно, что соискатели с большим опытом более 6 лет не слишком сильно представлены на платформе HH.ru. Скорее всего данные соискатели занимаюся поиском работы исходя из своих наработанных связей.*

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [53]:
# текст запроса

query_5_1 = f''' (SELECT e.name,
COUNT(v.id) -- находим количество вакансий по данному работодателю
FROM public.vacancies v
JOIN public.employers e ON v.employer_id = e.id -- соединяем таблицы вакансий и работодателей по ключевым полям
GROUP BY e.id-- группируем по имени работодателя
ORDER BY COUNT(v.id) DESC -- сортируем по количеству вакансий данного работадателя
LIMIT 1)
UNION ALL--объединяем таблицы
(SELECT e.name,
COUNT(v.id)
FROM public.vacancies v
JOIN public.employers e ON v.employer_id = e.id
GROUP BY e.id
ORDER BY  COUNT DESC
OFFSET 4 LIMIT 1)'''


In [54]:
# результат запроса

result = pd.read_sql_query(query_5_1, connection)
print(f"На первом месте по количеству вакансий находится - «{result.iloc[0, 0]}», а на пятом - «{result.iloc[1, 0]}»")



На первом месте по количеству вакансий находится - «Яндекс», а на пятом - «Газпром нефть»


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [55]:
# текст запроса

query_5_2 = f''' SELECT a.name,
COUNT(e.id) count_emp,
COUNT(v.id) count_vac
FROM public.areas a
JOIN public.employers e ON a.id = e.area -- соединяем таблицы регионов и работодателей
LEFT JOIN public.vacancies v ON a.id = v.area_id -- соединяем таблицы регионов и вакансий, чтобы сохранились регионы без вакансий
GROUP BY a.id --группируем по регионам
HAVING COUNT(v.id) = 0 -- находим регионы с отсутствующими вакансиями
ORDER BY COUNT(e.id) DESC
LIMIT 1 '''

In [56]:
# результат запроса

result = pd.read_sql_query(query_5_2, connection)
print('Наибольшее количество работодателей среди регионов, в которых нет вакансий, является: ', result.iloc[0, 0])


Наибольшее количество работодателей среди регионов, в которых нет вакансий, является:  Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [57]:
# текст запроса

query_5_3 = ''' SELECT e.name,
COUNT(DISTINCT v.area_id) -- находим количества только оригинальных названий регионов
FROM public.vacancies v
JOIN public.employers e ON v.employer_id = e.id
GROUP BY e.name
ORDER BY COUNT(DISTINCT v.area_id) DESC'''

In [58]:
# результат запроса

result = pd.read_sql_query(query_5_3, connection)
print(' Значение максимального количество регионов, в которых работодатель публикует свои вакансии, составляет: ', result.iloc[0, 1])
result

 Значение максимального количество регионов, в которых работодатель публикует свои вакансии, составляет:  181


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [59]:
# текст запроса
query_5_4 = f''' SELECT COUNT(e.id)
FROM public.employers e
LEFT JOIN public.employers_industries e_i ON e.id = e_i.employer_id -- соединяем таблицы работодателей и сфер деятельности, чтобы сохранились работодатели без сфер деятельности
WHERE e_i.industry_id IS NULL'''


In [60]:
# результат запроса

result = pd.read_sql_query(query_5_4, connection)
print(' Количество работодателей, у которых не указана сфера деятельности, составляет: ', result.iloc[0, 0])


 Количество работодателей, у которых не указана сфера деятельности, составляет:  8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [68]:
# текст запроса

query_5_5 = f''' SELECT e.name--,
--COUNT(e_i.industry_id)
FROM public.employers_industries e_i
JOIN public.employers e ON e_i.employer_id = e.id -- соединяем таблицы сфер деятельности и работодателей
GROUP BY e.id --группируем по id работодателя
HAVING COUNT(e_i.industry_id) = 4 -- фильтруем компании с 4-мя сферами деятельности
ORDER BY e.name ASC -- сортируем по алфавиту
OFFSET 2 LIMIT 1 -- сместиться на два значения и выбрать одно следующее значение'''

query_5_5_1 = f''' WITH Em_Count_Industries_of_Employers AS ( -- для круговой диаграммы создаем таблицу работодателей с количеством отмеченных у них сфер деятельности
SELECT e.name, 
COUNT(e_i.industry_id) Count_Industries_of_Employers
FROM public.employers_industries e_i
RIGHT JOIN public.employers e ON e_i.employer_id = e.id -- соединяем таблицы сфер деятельности и работодателей с незаполнеными строками сферы деятельности
GROUP BY e.id --группируем по id работодателя
--HAVING COUNT(e_i.industry_id) = 0 
ORDER BY e.name ASC) -- сортируем по алфавиту
SELECT Count_Industries_of_Employers,
COUNT(name)
FROM Em_Count_Industries_of_Employers
GROUP BY Count_Industries_of_Employers
ORDER BY Count_Industries_of_Employers ASC'''

In [69]:
# результат запроса

result = pd.read_sql_query(query_5_5, connection)
print(f"На 3-м месте в алфавитном списке находится компания «{result.iloc[0, 0]}», у которой указано четыре сферы деятельности.")

# Для лучшего понимания указывают ли работодатели сферу деятельности и количество отмеченных сфер деятельности, построим круговую диаграмму
import plotly.express as px

result_1 = pd.read_sql_query(query_5_5_1, connection)
result_1['percent'] = round(result_1['count']/(result_1['count'].sum())*100, 1)
labels = {0:'Не указана сфера деятельности', 1:'Указана 1 сфера деятельности', 2:'Указаны 2 сферы деятельности',3:'Указаны 3 сферы деятельности', 4:'Указаны 4 сферы деятельности', 5:'Указаны 5 сфер деятельности'}
result_1['count_industries_of_employers'] = result_1['count_industries_of_employers'].replace(labels)
fig = px.sunburst(data_frame=result_1, path=['count_industries_of_employers'], values='percent', width=615, height=700, 
                  title="Сколько сфер деятельности указывают компании-работадатели.")
                  #labels = {0:'Не указана сфера деятельности', 1:'Указана одна сфера деятельности', 2:'Указаны две сферы деятельности', 3:'Указаны три сферы деятельности', 4:'Указаны четыре сферы деятельности', 5:'Указаны пять сфер деятельности'})
                 # names=['Не указана сфера деятельности', 'Указана одна сфера деятельности', 'Указаны две сферы деятельности', 'Указаны три сферы деятельности', 'Указаны четыре сферы деятельности', 'Указаны пять сфер деятельности'])
fig.show("png")
#result_1

На 3-м месте в алфавитном списке находится компания «2ГИС», у которой указано четыре сферы деятельности.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [70]:
# текст запроса

query_5_6 = f''' SELECT
COUNT(e.id)
FROM public.employers_industries e_i
JOIN public.employers e ON e_i.employer_id = e.id -- соединяем таблицы сфер деятельности и работодателей
JOIN public.industries i ON e_i.industry_id = i.id-- соединяем таблицы сфер деятельности и расшифровки id сфер деятельности
WHERE i.name LIKE '%Разработка программного обеспечения%' '''

In [71]:
# результат запроса

result = pd.read_sql_query(query_5_6, connection)
print(f"{result.iloc[0, 0]} компании задействованы в сфере деятельности «Разработка программного обеспечения».")


3553 компании задействованы в сфере деятельности «Разработка программного обеспечения».


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [72]:
# код для получения списка городов-милионников

import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' #ссылка на статью из Wikipedia про города-миллионики в России
response = requests.get(url) #Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
blank_for_the_list =  page.select('table.standard.sortable > tbody > tr > td > a') # получаем обрезки строк (заготовку) из таблицы преобразованной через объект BeautifulSoup
#s = page.find_all('standard sortable') #(c find и find_all не работает)
list_black=list(map(str,blank_for_the_list)) # переводим каждый кусок строки в строковую переменную и помещаем в список

import re # вызываем библиотеку регулярных выражений

million_cities=[]
for elem in list_black:
    million_cities+= re.findall(r'>(\w*\s*\S*)<',elem) # суммируем два списка  million_cities(от предыдущей итерации) и список, полученный при отсечении всего ненужного из списка blank_for_the_list (из заготовки) по элементно  
str_million_cities = "'"+"', '".join(million_cities)+"'" # преобразуем список в строку

#million_cities_=pd.Series(data=million_cities, name='name_million_cities')
#print(str_million_cities)
#print(type(str_million_cities))
#print(len(million_cities)) 
#print(million_cities)   
#print(blank_for_the_list)
#print(list_black)
#print(page)

In [73]:
# текст запроса

query_5_7 = f''' SELECT a.name,
COUNT(v.id) count
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
JOIN public.employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс' AND a.name in ({str_million_cities})
GROUP BY a.id
UNION
SELECT 'Total', COUNT(v.id)
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
JOIN public.employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс' AND a.name in ({str_million_cities})
ORDER BY count ASC'''

In [74]:
# результат запроса

result = pd.read_sql_query(query_5_7, connection) 
result

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

***Комментарии и выводы:***

*На этапе анализа работадателей заметны следующие тенденции:*

- *Наибольшее количество вакансий отобранное в нашем конкретном перечне приходится на крупные компании: в IT сфере ("Яндекс", "Первый Бит"), в сфере телефонных и интернет коммуникаций ("Ростелеком", "МТС".), крупных банков("Тинькофф", "СБЕР"), электронных торговых площадок ("Ozon"), так же есть представители сырьевой отрасли("Газпром нефть", "Норникель").* 

- *Заметно, что вакансии строго привязаны к определенному месту (в основном конкретное место положение работадателя). В этом ничего удивительного нет, так как в основном собеседования проводят на территории работодателя, да и как уже отмечалось - преобладающая тендеция работодателей, что работники должны быть в офисе (или на территории работодателя). Тройка лидирующих регионов по числу работадателей разделяют следующие регионы: Москва, Санкт-Петербург, Минск.*

- *Значение максимального количество регионов, в которых работодатель публикует свои вакансии указывает нам на крупные компании, в которых компания осуществляет свою деятельность и в этих регионах есть представительства и широкая сеть филиалов. Такие крупные игроки в нашей базе данных: "Яндекс", "Ростелеком", "Спецремонт".*

- *На примере компании "Яндекс" можно проследить вполне объяснимую закономерность, о которой упониналось в выводах к детальному анализу об особенности рынка труда в больших и огромных городах (мегаполисах). Т.е. компания "Яндекс" сосредоточила свой поиск людей в основном в крупных регионах, так в России в городах-миллиониках поиск составил 25,1% от общего числа вакансий. Причем распределение количества вакансий по городам-миллиониках (это 16 городов) практически пропорционально численности этих городов.*

- *Из представленной выше диаграммы ("Сколько сфер деятельности указывают компании-работадатели.") видно, что у большого количество работадателей не указана ни одна сфера деятельность в процентном соотношении 35,8% (в абсолютном соотношении 8419 компании из 23501), указана одна сфера деятельности 27,5% (6452 компании из 23501) далее идет две сферы деятельности 15,5% (3639 из 23501).*

- *Только 15,1% (в абсолютном соотношении 3553 компании из 23501) компаний задействованы в сфере деятельности «Разработка программного обеспечения».*

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [77]:
# текст запроса

query_6_1 = f''' SELECT COUNT(v.id)
FROM public.vacancies v
WHERE LOWER(v.name) LIKE '%data%' OR LOWER(v.name) LIKE'%данн%' -- условие, что название вакансий должны содержать либо слово 'data', либо слово 'данн' в нижнем регистре''' 

In [78]:
# результат запроса

result = pd.read_sql_query(query_6_1, connection)
print('Число вакансий, имеющих отношение к данным, составляет', result.iloc[0, 0])

Число вакансий, имеющих отношение к данным, составляет 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [79]:
# текст запроса

query_6_2 = f''' SELECT COUNT(v.id)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' OR LOWER(v.name) LIKE '%data science%' OR LOWER(v.name) LIKE '%исследователь данных%'  -- условие, что название вакансий должны содержать либо слова 'data scientist', либо 'data science', либо 'исследователь данных' в нижнем регистре
    OR LOWER(v.name) LIKE '%machine learning%' OR LOWER(v.name) LIKE '%машинн%обучен%' OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%'))  -- условие, что название вакансий должны содержать либо слова 'machine learning', либо 'машинн%обучен%' в нижнем регистре, либо 'ML'(но не содержаться в сочетании 'HTML')
    AND (LOWER(v.name) LIKE '%junior%' OR LOWER(v.experience) LIKE '%нет опыта%' OR LOWER(v.employment) LIKE '%стажировка%') -- так же должен удовлетворять условию что название вакансий должны содержать либо слово 'junior', либо 'нет опыта', либо 'стажировка' в нижнем регистре '''

In [80]:
# результат запроса

result = pd.read_sql_query(query_6_2, connection)
print(f"Для начинающего специалиста уровня Junior предложена {result.iloc[0, 0]} вакансия")

Для начинающего специалиста уровня Junior предложена 51 вакансия


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [81]:
# текст запроса

query_6_3 = f''' SELECT COUNT(v.name)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' OR LOWER(v.name) LIKE '%data science%' OR LOWER(v.name) LIKE '%исследователь данных%' \
    OR LOWER(v.name) LIKE '%machine learning%' OR LOWER(v.name) LIKE '%машинн%обучен%' OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')) \
    AND (LOWER(v.key_skills) LIKE '%sql%' OR LOWER(v.key_skills) LIKE '%postgres%') -- условия для DS теже, только добавляются условия, что в ключевые навыки должны содержать либо слово 'sql', либо 'postgres' '''


In [82]:
# результат запроса

result = pd.read_sql_query(query_6_3, connection)
print('Для специалистов Data Science число вакансий, в которых в качестве ключевого навыка указан SQL или postgres, составляет:', result.iloc[0, 0])


Для специалистов Data Science число вакансий, в которых в качестве ключевого навыка указан SQL или postgres, составляет: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [83]:
# текст запроса

query_6_4 = f''' SELECT COUNT(v.name)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' OR LOWER(v.name) LIKE '%data science%' OR LOWER(v.name) LIKE '%исследователь данных%' \
    OR LOWER(v.name) LIKE '%machine learning%' OR LOWER(v.name) LIKE '%машинн%обучен%' OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')) \
    AND (LOWER(v.key_skills) LIKE '%python%') -- условия для DS теже, только добавляются условия, что в ключевые навыки должны содержать слово 'python' '''

In [84]:
# результат запроса

result = pd.read_sql_query(query_6_4, connection)
print('Для специалистов Data Science число вакансий, в которых в качестве ключевого навыка указан Python, составляет:', result.iloc[0, 0])

Для специалистов Data Science число вакансий, в которых в качестве ключевого навыка указан Python, составляет: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [85]:
# текст запроса

query_6_5 = f''' SELECT --*, --находим количество навыков в строке, исходя из того, что видим в ключевых навыках постоянно повторяющийся символ \t (TAB-табуляция (chr(9) - если через функцию совместимости)),
ROUND(AVG(LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills,'\t','')) + 1), 2) --т.е. разница количества символов в строке с \t и без него, но не забываем прибавить 1, т.к. у первого навыка нет \t
                                                                                -- далее берем среднее для всех строк и округляем до 2-х знаков
--LENGTH(v.key_skills) length,
--LENGTH(replace(v.key_skills,'\t','')) length_1

FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' OR LOWER(v.name) LIKE '%data science%' OR LOWER(v.name) LIKE '%исследователь данных%' \
    OR LOWER(v.name) LIKE '%machine learning%' OR LOWER(v.name) LIKE '%машинн%обучен%' OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')) -- условия для DS теже '''

In [86]:
# результат запроса

result = pd.read_sql_query(query_6_5, connection)
print(f"В среднем в вакансиях для специалистов Data Science указывают {result.iloc[0, 0]} ключевых навыков")

В среднем в вакансиях для специалистов Data Science указывают 6.41 ключевых навыков


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [87]:
# текст запроса

query_6_6 = f''' SELECT v.experience,
ROUND(AVG(CASE 
    WHEN v.salary_from IS NULL THEN v.salary_to
    WHEN v.salary_to IS NULL THEN v.salary_from
    ELSE (COALESCE(v.salary_from, 0) + COALESCE(v.salary_to, 0))/2 END), 0) AS salary
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' OR LOWER(v.name) LIKE '%data science%' OR LOWER(v.name) LIKE '%исследователь данных%' \
    OR LOWER(v.name) LIKE '%machine learning%' OR LOWER(v.name) LIKE '%машинн%обучен%' OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')) -- условия для DS теже 
    AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL ) -- условия, что нижняя граница зарплатной вилки или верхняя граница зарплатной вилки не должна быть пустой
GROUP BY v.experience '''


In [88]:
# результат запроса

result = pd.read_sql_query(query_6_6, connection)
print(f"Средняя зарплата у специалистов Data Science с опытом работы от 3 до 6 лет составляет: {result.iloc[2, 1]} руб.")
result

Средняя зарплата у специалистов Data Science с опытом работы от 3 до 6 лет составляет: 243115.0 руб.


,experience,salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

***Комментарии и выводы:***

*В предметном анализе можно отметить следующие моменты:*

- *Из всех вакансий, которые представлены в нашей базе данных, всего 1771 из 49197 вакансий имеют отношение к данным, в процентном соотношении это всего 3,6%. А для новичков без опыта работы и того меньше 0,1% от общего числа (в абсолютном соотношении 51 из 49197 вакансий) или 2,9% от числа вакансий связанных с данными (в абсолютном соотношении 51 из 1771 вакансии). Довольно узкий коридор возможностей.*

- *В вакансиях для специалистов Data Science указывают в основном чуть более шести ключевых навыков.*

- *Из этих навыков знание языка программирования Python является довольно нужным и распространенным. 19,8% из числа вакансий для специалистов Data Science (или 0,7% от общего числа вакансий). А так же довольно высоко ценится такой навык, как умениия работать в postgres и знание SQL. 11,3% из числа вакансий для специалистов Data Science (или 0,4% от общего числа вакансий).*

- *Зарплата у специалистов Data Science находятся на хорошем уровне и зависит от опыта работы, как в прочем и у многих, но прогресс возрастания зарплаты от первого этапа к следующему этапу довольно высокий, так разница между средними зарплатами у специалистов без опыта работы (74643 руб.) и с опытом работы от 3 до 6 лет (139675 руб.) составляет - 65032 руб. (это на 87% больше чем на первой ступени зарплат), но и следующая разница между средними зарплатами у специалистов с опытом работы от 3 до 6 лет и от 3 до 6 лет (243115 руб.) впечатляет и составляет 103440 руб. (что на 74% больше чем на предыдущей ступени зарплат).*

# Общий вывод по проекту

In [5]:

# В заключении хочется узнать какие сферы деятельности наиболее нуждаются в работниках (в нашей базе данных) и на сколько опытные сотрудники нужны в данных сферах деятельности

# текст запроса
query_conclusion = f''' WITH Popular_Industries AS (SELECT i.name name_popular_industries,
--v.experience,
COUNT(v.id) count_vac_of_pup_ind
FROM public.industries i
JOIN public.employers_industries e_i ON i.id = e_i.industry_id
JOIN public.employers e ON e_i.employer_id = e.id
JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY i.name
ORDER BY COUNT(v.id) DESC
LIMIT 8)
SELECT p_i.name_popular_industries,
p_i.count_vac_of_pup_ind,
v.experience,
COUNT(v.id) count_vac
FROM Popular_Industries p_i 
JOIN public.industries i ON p_i.name_popular_industries = i.name
JOIN public.employers_industries e_i ON i.id = e_i.industry_id
JOIN public.employers e ON e_i.employer_id = e.id
JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY p_i.name_popular_industries, p_i.count_vac_of_pup_ind, v.experience
ORDER BY p_i.count_vac_of_pup_ind DESC, count_vac DESC '''


In [6]:
# результат запроса

result_conclusion = pd.read_sql_query(query_conclusion, connection)
#display(result_conclusion)

import plotly.express as px
# Построим диаграмму на основании полученных данных из запроса 
fig = px.bar(data_frame=result_conclusion, x='experience', y='count_vac', color='name_popular_industries', barmode='group', 
             #text='name_popular_industries', 
             title="Рейтинг потребности в сотрудниках с разным опытом работы в сферах деятельности, в которых задействованы компании-работодатели.",
             labels={'experience':'Опыт работы, годы', 'count_vac':'Количество вакансий'}, height =600, width = 1400)
#fig.update_layout(uniformtext_minsize=14, uniformtext_mode='hide', title_xanchor='center', title_x=0.5)  # настраиваем подпись на столбцах диаграммы и заголовок диаграммы
fig.update_layout( legend=dict( font = dict(size= 12), borderwidth=0.5, orientation= 'h', x= 0.5, y=- 0.4,  
                               xanchor= 'center', yanchor= 'bottom', title= 'Сферы деятельности:')) # настраиваем легенду
fig.show("png")

connection.close()

***Общие комментарии и выводы:***

*На основании проведенных исследований по завершению всех этапов анализа можно отметить следующие:*

- *В базе HH.ru накоплен обширный диапазон регионов, компаний-работодателей, сфер их деятельности, но временной период (интервал времени) скорее всего взят небольшой этим объясняется не столь большой объем вакансий (всего 2 с небольшим раза больше количества работодателей).*

- *Как видно из диаграммы наибольшую потребность в сотрудниках имеют компании-работодатели занятые в сферах деятельности такие, как: разработка программного обеспечения; системная интеграции, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг; интернет-компания(поисковики, платежные системы, социальные сети и прочее); банки, мобильная связь. Причем таже тенденция сохраняется практически не зависимо от опыта работы, отличие в последовательности наблюдаются лишь для соискателей без опыта работы, т.е. потребность в сотрудниках даже без опыта работы сохраняется в интернет-компаниях; в компаниях занимающихся разработкой программного обеспечения; системной интеграцией, автоматизацией технологических и бизнес-процессов предприятия, ИТ-консалтингом; мобильной связью и далее банки.* 

 - *Наибольшее количество вакансий представлена с опытом работы от 1 года до 3 лет. Видно, что соискатели с большим опытом более 6 лет не слишком сильно представлены на платформе HH.ru. Скорее всего данные соискатели занимаюся поиском работы исходя из своих наработанных связей.*

- *На протяжении всего исследования (на разных этапах и в различных примерах) прослеживается вполне объяснимая закономерность, особенность рынка труда, так в больших и огромных городах (мегаполисах) существует довольно большое количество предложений по трудоустройству (большое количество разных предприятий, разных представительств крупных фирм, торговых площадок, тц, большое количество людей в сфере обслуживания и т.д.), существует конкуренция между работодателями, а у работника есть выбор кому продать свои знания и умения подороже. Также жить в мегаполисе довольно дорогое удовольствие, поэтому и заработная плата нужна побольше, чем в небольших городах.*

- *Следует отметить, что только 3553 компании из 23501, что составляет 15,1% компаний задействованы в сфере деятельности «Разработка программного обеспечения». И всего 3,6% от общего числа вакансий имеет отношение к данным, а для новичков без опыта работы и того меньше 0,1% от общего числа (в абсолютном соотношении 51 из 49197 вакансий) или 2,9% от числа вакансий связанных с данными (в абсолютном соотношении 51 из 1771 вакансии),что является довольно узким коридором возможностей.*

- *В вакансиях для специалистов Data Science указывают в основном чуть более шести ключевых навыков. Из этих навыков знание языка программирования Python является довольно нужным и распространенным. 19,8% из числа вакансий для специалистов Data Science (или 0,7% от общего числа вакансий). А так же довольно высоко ценится такой навык, как умениия работать в postgres и знание SQL. 11,3% из числа вакансий (или 0,4% от общего числа вакансий).*

- *Зарплата у специалистов Data Science находятся на хорошем уровне и зависит от опыта работы, как в прочем и у многих, но прогресс возрастания зарплаты от первого этапа к следующему этапу довольно высокий, так разница между средними зарплатами у специалистов без опыта работы (74643 руб.) и с опытом работы от 3 до 6 лет (139675 руб.) составляет - 65032 руб. (это на 87% больше чем на первой ступени зарплат), но и следующая разница между средними зарплатами у специалистов с опытом работы от 3 до 6 лет и от 3 до 6 лет (243115 руб.) впечатляет и составляет 103440 руб. (что на 74% больше чем на предыдущей ступени зарплат).*

*Для уточнения данных и лучшего понимания динамики ситуации на рынке труда необходимы дополнительные данные: изменение данного процесса во временном периоде. А именно нужно проследить как повлияли на рынок труда санкции, кризис, уход инвестиций. Все это поможет осознать, что происходит на текущей момент, и составить возможный прогноз каким образом может развиваться ситуация на рынке труда (причем по возможности в нескольких вариантах). Это также даст возможность лучше проанализировать прогресс развития наших городов и в целом регионов.*